## Australian Rainfall Data Download and Exploratory Data Analysis

Attribution: code adapted from DSCI 525 lecture notes

#### 1. Imports 

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

#### 2. Downloading data using figshare API

In [2]:
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.41 s, sys: 6.79 s, total: 12.2 s
Wall time: 3min 3s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18.1 s, sys: 3.36 s, total: 21.5 s
Wall time: 22.6 s


#### 3. Combining data files into one CSV file

In [8]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(file_name=re.findall(r"[\/|\\](.*)_daily_rainfall", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 5min 51s, sys: 24.3 s, total: 6min 15s
Wall time: 6min 32s


In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


In [10]:
%%time
df.head()

CPU times: user 184 µs, sys: 528 µs, total: 712 µs
Wall time: 720 µs


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),file_name
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


##### 3.1 Comparing run times on different machines 

- We used pandas to concatenate the files in data.zip folder to a combined file which is 5.6GB in size.
- ** ADD A GENERAL OBSERVATION OF RUN TIMES AMONG THE GROUP MEMBERS**
- ** If anyone runs into issue, specify here...**

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to combine files | Time taken to read files |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
| Arlin   |      MacOS Monterey            |  16GB   | 2 GHz Quad-Core Intel Core i5          |    No    |   CPU Time: 5 min 52s; Total: 6 min 9s; Wall time: 6 min 20s|CPU Time: 87µs; Total: 88 µs; Wall time: 90.8 µs|
| Member 2    |                  |     |           |        |            |          |
| Member 3    |                  |     |           |        |            |          |


#### 4. Load the combined CSV to memory and perform a simple EDA